1. Setup & Extract Zip

In [8]:
# ===========================
# 1. BASIC SETUP & ZIP EXTRACT
# ===========================
import os
import zipfile
import numpy as np
import pandas as pd
import random

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import layers, models
import joblib

# For reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ---- CONFIG: EDIT THESE IF NEEDED ----
DATASET_ZIP = "/content/v2.zip"  # <- name of your uploaded zip
EXTRACT_DIR = "/content"         # where to extract
DATASET_DIR  = os.path.join(EXTRACT_DIR, "v2")  # root of v2 data

# If your zip structure is different, print after extract and adjust DATASET_DIR

# ---- Extract zip (run once per Colab session) ----
if not os.path.exists(EXTRACT_DIR):
    os.makedirs(EXTRACT_DIR, exist_ok=True)

with zipfile.ZipFile(DATASET_ZIP, "r") as z:
    z.extractall(EXTRACT_DIR)

print("Extracted contents:")
for root, dirs, files in os.walk(EXTRACT_DIR):
    print(root)
    # only show first level or two
    break


Extracted contents:
/content


2. Data Loading Helpers

In [10]:
# ===========================
# 2. DATA LOADING FUNCTIONS
# ===========================

# column names in each txt file
COLS = ["t", "emg1", "emg2", "emg3", "ax", "ay", "az", "gx", "gy", "gz"]
FEATURE_COLS = ["emg1", "emg2", "emg3", "ax", "ay", "az", "gx", "gy", "gz"]

def load_sensor_file(path):
    """
    Load a single .txt file into a DataFrame.
    Handles files without headers: comma-separated numeric values.
    """
    # robust reading (if there are any weird bytes, latin-1 won't crash)
    df = pd.read_csv(
        path,
        header=None,
        names=COLS,
        encoding="latin-1",
        on_bad_lines='skip' # Added to skip lines with too many columns
    )

    # drop completely empty rows (if any)
    df = df.dropna(how="all")

    # keep only numeric columns, coerce errors
    for c in COLS:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna().reset_index(drop=True)

    return df


def load_dataset(dataset_dir):
    """
    Walks dataset_dir and loads all gesture samples.
    Assumes:
        dataset_dir / <gesture_name> / Day_* / *.txt
    Returns:
        X_raw: list of (Ti, num_features) numpy arrays
        y_raw: list of gesture labels (strings)
        gesture_list: sorted list of unique gesture names
    """
    X_raw = []
    y_raw = []

    # gesture folders (e.g. ada, awidinawa, boru, ...)
    gesture_dirs = sorted([
        d for d in os.listdir(dataset_dir)
        if os.path.isdir(os.path.join(dataset_dir, d))
    ])

    print("Found gesture folders:", gesture_dirs)

    for gesture in gesture_dirs:
        g_path = os.path.join(dataset_dir, gesture)

        # inside each gesture, expect Day_1, Day_2, ... or txts directly
        # handle both patterns
        subdirs = [
            os.path.join(g_path, d) for d in os.listdir(g_path)
            if os.path.isdir(os.path.join(g_path, d))
        ]
        txt_files = [
            os.path.join(g_path, f) for f in os.listdir(g_path)
            if f.endswith(".txt")
        ]

        # case 1: Day subfolders
        if subdirs:
            for day_dir in sorted(subdirs):
                for fname in sorted(os.listdir(day_dir)):
                    if not fname.endswith(".txt"):
                        continue
                    fpath = os.path.join(day_dir, fname)
                    df = load_sensor_file(fpath)
                    if len(df) == 0:
                        continue
                    X_raw.append(df[FEATURE_COLS].values)
                    y_raw.append(gesture)
        # case 2: txt files directly in gesture folder
        if txt_files:
            for fpath in sorted(txt_files):
                df = load_sensor_file(fpath)
                if len(df) == 0:
                    continue
                X_raw.append(df[FEATURE_COLS].values)
                y_raw.append(gesture)

    gesture_list = sorted(list(set(y_raw)))
    return X_raw, y_raw, gesture_list


# ---- Load everything ----
X_raw, y_raw, gesture_list = load_dataset(DATASET_DIR)
print("Total samples:", len(X_raw))
print("Classes:", gesture_list)

# Quick check of sequence lengths
lengths = [x.shape[0] for x in X_raw]
print("Min len:", min(lengths), "Max len:", max(lengths), "Median:", int(np.median(lengths)))

Found gesture folders: ['ada', 'awidinawa', 'boru', 'hawasa', 'hodai', 'irida', 'narakai', 'pata', 'saduda', 'udasana']
Total samples: 1000
Classes: ['ada', 'awidinawa', 'boru', 'hawasa', 'hodai', 'irida', 'narakai', 'pata', 'saduda', 'udasana']
Min len: 400 Max len: 1070 Median: 680


3. Preprocessing: Normalization + Padding

In [11]:
# ===========================
# 3. PREPROCESSING
# ===========================

MAX_LEN = 600  # you can tune this (e.g. 512, 800, ...)

num_features = len(FEATURE_COLS)

# --- 3.1 Fit scaler on all data (stack all frames) ---
all_frames = np.vstack(X_raw)  # (total_frames, num_features)
scaler = StandardScaler()
scaler.fit(all_frames)
print("Scaler fitted on frames:", all_frames.shape)

# --- 3.2 Helper to scale + pad/truncate ---
def preprocess_sequence(seq):
    """
    seq: (T, num_features)
    returns: (MAX_LEN, num_features) float32
    """
    seq_scaled = scaler.transform(seq)

    if len(seq_scaled) > MAX_LEN:
        seq_scaled = seq_scaled[:MAX_LEN]
    else:
        pad_len = MAX_LEN - len(seq_scaled)
        pad = np.zeros((pad_len, num_features), dtype=np.float32)
        seq_scaled = np.vstack([seq_scaled, pad])

    return seq_scaled.astype(np.float32)


# --- 3.3 Encode labels ---
label_to_idx = {label: i for i, label in enumerate(gesture_list)}
idx_to_label = {i: l for l, i in label_to_idx.items()}
print("Label map:", label_to_idx)

# --- 3.4 Apply preprocessing to all samples ---
X = np.stack([preprocess_sequence(seq) for seq in X_raw])  # (N, MAX_LEN, num_features)
y = np.array([label_to_idx[l] for l in y_raw], dtype=np.int64)

print("X shape:", X.shape, "y shape:", y.shape)


Scaler fitted on frames: (690989, 9)
Label map: {'ada': 0, 'awidinawa': 1, 'boru': 2, 'hawasa': 3, 'hodai': 4, 'irida': 5, 'narakai': 6, 'pata': 7, 'saduda': 8, 'udasana': 9}
X shape: (1000, 600, 9) y shape: (1000,)


4. Train / Val / Test Split

In [12]:
# ===========================
# 4. TRAIN / VAL / TEST SPLIT
# ===========================
test_size = 0.15
val_size = 0.15  # of remaining train

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=test_size, random_state=SEED, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=val_size, random_state=SEED, stratify=y_trainval
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Train: (722, 600, 9) Val: (128, 600, 9) Test: (150, 600, 9)


5. CNN + LSTM Model

In [13]:
# ===========================
# 5. BUILD CNN + LSTM MODEL
# ===========================
num_classes = len(gesture_list)

def build_cnn_lstm_model(max_len, num_features, num_classes):
    inputs = layers.Input(shape=(max_len, num_features))

    x = layers.Conv1D(64, kernel_size=5, padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(128, kernel_size=5, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = build_cnn_lstm_model(MAX_LEN, num_features, num_classes)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 600, 9)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 600, 64)        │         2,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 600, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 300, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 300, 128)       │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 150, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 150, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 75, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,314 (790.29 KB)

 Trainable params: 201,674 (787.79 KB)

 Non-trainable params: 640 (2.50 KB)

6. Train With Callbacks

In [14]:
# ===========================
# 6. TRAINING
# ===========================
batch_size = 32
epochs = 50

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=8,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=4,
    min_lr=1e-5,
    verbose=1
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 13s 273ms/step - accuracy: 0.1584 - loss: 2.3067 - val_accuracy: 0.2578 - val_loss: 2.2188 - learning_rate: 0.0010
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 328ms/step - accuracy: 0.3169 - loss: 1.8822 - val_accuracy: 0.2969 - val_loss: 2.0933 - learning_rate: 0.0010
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 244ms/step - accuracy: 0.3973 - loss: 1.6761 - val_accuracy: 0.3594 - val_loss: 1.9942 - learning_rate: 0.0010
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 290ms/step - accuracy: 0.4565 - loss: 1.5287 - val_accuracy: 0.3359 - val_loss: 1.8528 - learning_rate: 0.0010
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 252ms/step - accuracy: 0.5245 - loss: 1.4131 - val_accuracy: 0.4062 - val_loss: 1.7198 - learning_rate: 0.0010
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 328ms/step - accuracy: 0.5315 - loss: 1.2794 - val_accuracy: 0.4297 - val_loss: 1.5039 - learning_rate: 0.0010
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 243ms/step - accuracy: 0.5769 - loss: 1.1647 -

7. Evaluation on Test Set

In [15]:
# ===========================
# 7. EVALUATION
# ===========================

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")

# Detailed report
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=gesture_list))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Test accuracy: 0.9333, Test loss: 0.2012
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 379ms/step

Classification Report:
              precision    recall  f1-score   support

         ada       0.93      0.87      0.90        15
   awidinawa       0.93      0.93      0.93        15
        boru       0.93      0.93      0.93        15
      hawasa       1.00      0.87      0.93        15
       hodai       0.88      1.00      0.94        15
       irida       0.83      1.00      0.91        15
     narakai       1.00      0.87      0.93        15
        pata       1.00      1.00      1.00        15
      saduda       1.00      0.93      0.97        15
     udasana       0.88      0.93      0.90        15

    accuracy                           0.93       150
   macro avg       0.94      0.93      0.93       150
weighted avg       0.94      0.93      0.93       150

Confusion Matrix:
[[13  0  0  0  1  1  0  0  0  0]
 [ 1 14  0  0  0  0  0  0  0  0]
 [ 0  0 14  0  1  0  0  0  0  0]
 [ 0  0  0 13  0  0 

8. Save Model, Scaler, and Label Map

In [16]:
# ===========================
# 8. SAVE ARTIFACTS
# ===========================
MODEL_PATH = "/content/gesture_model_v2.h5"
SCALER_PATH = "/content/scaler_v2.pkl"
LABEL_MAP_PATH = "/content/label_map_v2.pkl"

model.save(MODEL_PATH)
joblib.dump(scaler, SCALER_PATH)
joblib.dump(label_to_idx, LABEL_MAP_PATH)

print("Saved model to:", MODEL_PATH)
print("Saved scaler to:", SCALER_PATH)
print("Saved label map to:", LABEL_MAP_PATH)


Saved model to: /content/gesture_model_v2.h5
Saved scaler to: /content/scaler_v2.pkl
Saved label map to: /content/label_map_v2.pkl


In [17]:
from google.colab import files
files.download(MODEL_PATH)
files.download(SCALER_PATH)
files.download(LABEL_MAP_PATH)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>